[![](https://img.shields.io/badge/Source%20on%20GitHub-orange)](https://github.com/laminlabs/lamin-spatial/blob/main/docs/vitessce.ipynb) [![hub](https://img.shields.io/badge/View%20on%20lamin.ai-mediumseagreen)](https://lamin.ai/vitessce/examples/transform/BZhZQ6uIbkWv)

# Vitessce: AnnData

This tutorial demonstrates how to use Vitessce to create interactive visualizations for data stored in LaminDB artifacts. 
It has been adapted from the data preparation examples in [the Vitessce documentation](https://vitessce.github.io/vitessce-python).

In this part, we'll visualize an AnnData object stored in both H5AD and Zarr formats.

In [ ]:
# pip install "vitessce[all]>=3.5.0" "generate-tiff-offsets>=0.1.9" lamindb
!lamin connect laminlabs/lamin-dev  # <-- replace with your instance

In [ ]:
import time

import vitessce as vit
from vitessce import data_utils as vitdu
import lamindb as ln
import json

ln.track()

## Visualize an AnnData object (H5AD format)

Here we use a dataset from the [COVID-19 Cell Atlas](https://www.covid19cellatlas.org/index.healthy.html#habib17) that has been previously subset to highly variable genes and ingested into the public [vitessce/examples](https://lamin.ai/vitessce/examples) instance.

Create the central query object for this instance:

In [ ]:
db = ln.DB("vitessce/examples")

In [ ]:
h5ad_artifact = db.Artifact.get(key="vitessce_examples/habib17.h5ad").save()

When using `.h5ad` files, we construct a [Reference Specification](https://fsspec.github.io/kerchunk/spec.html) which enables interoperability with the [Zarr](https://zarrita.dev/packages/storage.html#referencestore) interface.

In [ ]:
h5ad_artifact.load().write("habib17.h5ad")

ref_spec = vitdu.generate_h5ad_ref_spec("habib17.h5ad")

Next, we need to save the corresponding Reference Specification to a JSON file and upload it as an artifact.

In [ ]:
ref_path = "./habib17.processed.reference.json"

with open(ref_path, "w") as file:
    json.dump(ref_spec, file)

ref_artifact = ln.Artifact(
    ref_path,
    key="vitessce_examples/habib17.reference.json",
    description="Reference JSON for H5AD file, Habib et al., 2017 Nature Methods",
).save()

### Save a VitessceConfig object

You can create a dashboard for one or several datasets by using Vitessce's component API.

You can pass artifacts to the `AnnDataWrapper` class using the `adata_artifact` and `ref_artifact` [parameters](https://python-docs.vitessce.io/api_data.html#vitessce.wrappers.AnnDataWrapper).

In [ ]:
vc = vit.VitessceConfig(
    schema_version="1.0.18",
    description=h5ad_artifact.description,
)

dataset = vc.add_dataset(name="Habib 2017").add_object(
    vit.AnnDataWrapper(
        adata_artifact=h5ad_artifact,
        ref_artifact=ref_artifact,
        obs_feature_matrix_path="X",
        obs_embedding_paths=["obsm/X_umap"],
        obs_embedding_names=["UMAP"],
        obs_set_paths=["obs/CellType"],
        obs_set_names=["Cell Type"],
    )
)

obs_sets = vc.add_view(vit.ViewType.OBS_SETS, dataset=dataset)
obs_sets_sizes = vc.add_view(vit.ViewType.OBS_SET_SIZES, dataset=dataset)
scatterplot = vc.add_view(vit.ViewType.SCATTERPLOT, dataset=dataset, mapping="UMAP")
heatmap = vc.add_view(vit.ViewType.HEATMAP, dataset=dataset)
genes = vc.add_view(vit.ViewType.FEATURE_LIST, dataset=dataset)
vc.link_views([scatterplot, heatmap], ["featureValueColormapRange"], [[0.0, 0.1]])
vc.layout(((scatterplot | obs_sets) / heatmap) | (obs_sets_sizes / genes))

Save the `VitessceConfig` object.

In [ ]:
h5ad_vc_artifact = ln.integrations.save_vitessce_config(
    vc,
    description="View Habib17 (h5ad) in Vitessce",
)

In [ ]:
h5ad_vc_artifact.view_lineage()

In [ ]:
# compare the generated vitessce config to the public one
public_vc_json = db.Artifact.get("ffUKrGJGNHL3TDhG0000").load()
h5ad_vc_json = h5ad_vc_artifact.load()

assert public_vc_json["layout"] == h5ad_vc_json["layout"]
assert public_vc_json["coordinationSpace"] == h5ad_vc_json["coordinationSpace"]

:::{note}

After running `save_vitessce_config`, a Vitessce button will appear next to the dataset in the [Artifacts](https://lamin.ai/vitessce/examples/artifacts) page of the web interface.

If your `VitessceConfig` object references data from multiple artifacts, the Vitessce button will appear next to a `Collection` that groups these artifacts (on the [Collections](https://lamin.ai/vitessce/examples/collections) tab of the Artifacts page).
Note that when using an `.h5ad`-based Artifact, the presence of the corresponding `.reference.json` file will result in the creation of a Collection.

:::

## Visualize an AnnData object (Zarr format)

AnnData objects can be saved on-disk to not only `.h5ad` files, but also to [Zarr stores](https://zarr.readthedocs.io/en/stable/tutorial.html#storage-alternatives) using AnnData's [write_zarr](https://anndata.readthedocs.io/en/latest/generated/anndata.AnnData.write_zarr.html) method.

Just like in the above section, we use a zarr storage that has been previously written with `write_zarr()` and subset to highly variable genes and ingested into the `vitessce/examples` instance.

In [ ]:
adata_zarr_artifact = db.Artifact.get(key="vitessce_examples/habib17.adata.zarr").save()

### Save a VitessceConfig object

You can create a dashboard for one or several datasets by using Vitessce's component API.
Here, we configure the visualization the same way as above in the `.h5ad`-based example, with the exception of the `ref_artifact` parameter, as `.zarr`-based AnnData objects do not require a Reference Specification for Zarr interoperability.

In [ ]:
vc = vit.VitessceConfig(
    schema_version="1.0.18",
    description=adata_zarr_artifact.description,
)

dataset = vc.add_dataset(name="Habib 2017").add_object(
    vit.AnnDataWrapper(
        adata_artifact=adata_zarr_artifact,
        obs_feature_matrix_path="X",
        obs_embedding_paths=["obsm/X_umap"],
        obs_embedding_names=["UMAP"],
        obs_set_paths=["obs/CellType"],
        obs_set_names=["Cell Type"],
    )
)

obs_sets = vc.add_view(vit.Component.OBS_SETS, dataset=dataset)
obs_sets_sizes = vc.add_view(vit.Component.OBS_SET_SIZES, dataset=dataset)
scatterplot = vc.add_view(vit.Component.SCATTERPLOT, dataset=dataset, mapping="UMAP")
heatmap = vc.add_view(vit.Component.HEATMAP, dataset=dataset)
genes = vc.add_view(vit.Component.FEATURE_LIST, dataset=dataset)

vc.link_views([scatterplot, heatmap], ["featureValueColormapRange"], [[0.0, 0.1]])
vc.layout(((scatterplot | obs_sets) / heatmap) | (obs_sets_sizes / genes))

Save the `VitessceConfig` object.

In [ ]:
adata_zarr_vc_artifact = ln.integrations.save_vitessce_config(
    vc,
    description="View Habib17 in Vitessce",
)

In [ ]:
adata_zarr_vc_artifact.view_lineage()

In [ ]:
# compare the generated vitessce config to the public one
public_vc_json = db.Artifact.get("J4tMB6qAeHvsgEsp0000").load()
adata_zarr_vc_json = adata_zarr_vc_artifact.load()

assert public_vc_json["layout"] == adata_zarr_vc_json["layout"]
assert public_vc_json["coordinationSpace"] == adata_zarr_vc_json["coordinationSpace"]

This will render an interactive Vitessce viewer:

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/6bYCczExolBzBiQH0002.png" width="900px">

In [ ]:
ln.finish()

In [ ]:
ln.Collection.get(key="View Habib17 (h5ad) in Vitessce").delete(permanent=True)
ref_artifact.delete(permanent=True)
h5ad_artifact.delete(permanent=True, storage=False)
h5ad_vc_artifact.delete(permanent=True)

adata_zarr_artifact.delete(permanent=True, storage=False)
adata_zarr_vc_artifact.delete(permanent=True)